In [15]:
import numpy as np
import pandas as pd
from netCDF4 import Dataset
import state_locations

In [18]:
#### Loads MERRA netcdf file and important variables ####
# Note: operates over one day of data based on the daily nature of each MERRA file # 
def organize_MERRAvar(MERRA_file_path):
    data = Dataset(MERRA_file_path)
    
    ### Extracts location variables and other important variables from "data" ###
    lons = data.variables['lon'][:]
    lats = data.variables['lat'][:]
    hurs = data.variables['hurs'][:,:,:]
    
    ### print the variables saved on the dataset
    #print(data.variables)
    
    ### Defines time variable ###
    time = np.arange(1,hurs.shape[0]+1)
    
    
    ### Consolidates time, location, and variables of interest (i.e. temp) into one dataframe ###
    names = ['t','y','x']
    index = pd.MultiIndex.from_product([time,lats,lons], names=names)
    df = pd.DataFrame({'hurs': hurs.flatten()}, index=index)['hurs']
    df.index.names = ['time', 'lats', 'lons']
    df = df.reset_index(level=['time', 'lats', 'lons'])
    print(df)
    #df = pd.DataFrame.to_numpy(df)
    
    W, E, S, N = state_locations.get_state_loc('IN')
    
    INDIANA= df[((df["lats"]> W)&(df["lats"]< E))|((df["lons"]> S)&(df["lons"]< N))]
    
    print(INDIANA)
    
    # Adds temperature columns (one for each timestamp) # 
    #nlons = lons.shape[0]
    #nlats = lats.shape[0]
    #blockt0 = nlons*nlats
    
    
    
    
    # Create temp matrix with the columns for each time + appends to df_new #
    #df_new = df[0:blockt0,1:3]
    #for count, value in enumerate(time,start=0):    
    #    t_append = df[(count)*blockt0:(count+1)*blockt0,3]
    #    df_new = np.column_stack((df_new,t_append))
    
    #print(df_new)
    
    #return df_new, time, lats, lons, nlons

In [19]:
organize_MERRAvar("/Users/inunezfe/Documents/hurs_day_HadGEM2-ES_rcp45_r1i1p1_EWEMBI_20060101-20101231.nc4")

           time   lats    lons       hurs
0             1  89.75 -179.75  100.00000
1             1  89.75 -179.25  100.00000
2             1  89.75 -178.75  100.00000
3             1  89.75 -178.25  100.00000
4             1  89.75 -177.75  100.00000
...         ...    ...     ...        ...
473299195  1826 -89.75  177.75   91.79541
473299196  1826 -89.75  178.25   91.79541
473299197  1826 -89.75  178.75   91.79541
473299198  1826 -89.75  179.25   91.79541
473299199  1826 -89.75  179.75   91.79541

[473299200 rows x 4 columns]
           time   lats   lons       hurs
435           1  89.75  37.75  100.00000
436           1  89.75  38.25  100.00000
437           1  89.75  38.75  100.00000
438           1  89.75  39.25  100.00000
439           1  89.75  39.75  100.00000
...         ...    ...    ...        ...
473298919  1826 -89.75  39.75   91.79541
473298920  1826 -89.75  40.25   91.79541
473298921  1826 -89.75  40.75   91.79541
473298922  1826 -89.75  41.25   91.79541
473298923  1826

In [8]:
### coordinates from locations are in the following order: W, E, S, N
W, E, S, N = state_locations.get_state_loc('IN')
    
print(W,E,S,N)

-88.125 -84.375 37.5 42.0


In [10]:
-84 > -88

True